In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Load CSV
df = pd.read_csv('test_final_cohort.csv')

# Save as Parquet
df.to_parquet('mimic_cohort.parquet')

lca_df = pd.read_csv('lca_with_classes.csv')

lca_df.to_parquet('lca.parquet')

In [ ]:
mimic_cohort = pd.read_parquet('mimic_cohort.parquet')
mimic_cohort.head()

In [ ]:
lca_cohort = pd.read_parquet('lca.parquet')
lca_cohort.head()

In [ ]:
elixhauser = pd.read_csv("elixhauser_ahrq_no_drg_filter.csv")

elixhauser.to_parquet("elixhauser.parquet")

In [ ]:
elixhauser_parquet = pd.read_parquet("elixhauser.parquet")
elixhauser_parquet.head()

In [ ]:
comorbidity_cols = [
    'congestive_heart_failure',
    'cardiac_arrhythmias',
    'valvular_disease',
    'pulmonary_circulation',
    'peripheral_vascular',
    'hypertension',
    'paralysis',
    'other_neurological',
    'chronic_pulmonary',
    'diabetes_uncomplicated',
    'diabetes_complicated',
    'hypothyroidism',
    'renal_failure',
    'liver_disease',
    'peptic_ulcer',
    'aids',
    'lymphoma',
    'metastatic_cancer',
    'solid_tumor',
    'rheumatoid_arthritis',
    'coagulopathy',
    'obesity',
    'weight_loss',
    'fluid_electrolyte',
    'blood_loss_anemia',
    'deficiency_anemias',
    'alcohol_abuse',
    'drug_abuse',
    'psychoses',
    'depression'
]
X = elixhauser_parquet[comorbidity_cols].values
X

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Age group binning for each patient
age_bins = [16, 25, 45, 65, 85, 100]
age_labels = ['16-24', '25-44', '45-64', '65-84', '85-100']
elixhauser_parquet['age_group'] = pd.cut(elixhauser_parquet['age_years'], bins=age_bins, labels=age_labels, right=False)

# Prevalence matrix: Index = age_group, Columns = comorbidities
prevalence_by_age = elixhauser_parquet.groupby('age_group', observed=False)[comorbidity_cols].mean()
# This gives you the table: each cell is prevalence of a comorbidity in that age group
n_clusters = 6  # Set to number of clusters used in the paper
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X)
elixhauser_parquet['cluster'] = cluster_labels


In [ ]:
# Compute the mean presence of each comorbidity feature in each cluster
cluster_summary = elixhauser_parquet.groupby('cluster')[comorbidity_cols].mean()
print(cluster_summary)


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Build the graph
G = nx.Graph()
for comorb in comorbidity_cols:
    G.add_node(comorb)

# Filter data for the cluster to analyze
df_cluster = elixhauser_parquet

# Calculate prevalence for nodes (simple fraction having disease)
N = len(df_cluster)
prevalence = {comorb: df_cluster[comorb].sum() / N for comorb in comorbidity_cols}

# --- EDGE CONSTRUCTION WITH RR and 99% CONFIDENCE INTERVAL ---
for i, disease_a in enumerate(comorbidity_cols):
    for disease_b in comorbidity_cols[i+1:]:
        C_AB = ((df_cluster[disease_a]==1) & (df_cluster[disease_b]==1)).sum()
        P_A = prevalence[disease_a]
        P_B = prevalence[disease_b]
        if C_AB > 0 and P_A > 0 and P_B > 0:
            RR = (C_AB / N) / (P_A * P_B)
            sigma = 1/C_AB + 1/(P_A * P_B * N**2)
            lower = RR * np.exp(-1.96 * sigma)
            upper = RR * np.exp(1.96 * sigma)
            if not (lower <= 1 <= upper):
                G.add_edge(disease_a, disease_b, weight=RR)
                
# Prepare node sizes and caps for better visualization
scale_factor = 1.5
node_sizes = [prevalence[n] * 2000 for n in G.nodes]  # tune factor for best visual effect

# Prepare edge widths and styles
edge_weights = [G[u][v]['weight'] for u, v in G.edges]
min_weight = min(edge_weights) if edge_weights else 0
max_weight = max(edge_weights) if edge_weights else 1  # avoid div zero

edge_colors = []
edge_widths = []
edge_styles = []

for u, v in G.edges:
    w = G[u][v]['weight']
    width = 1 + 4 * (w - min_weight) / (max_weight - min_weight)  # scale width 1-5
    edge_widths.append(width)
    if w < 2.5:  # Dashed for modest RR, solid for strong (adjust threshold as needed)
        edge_styles.append('dashed')
        edge_colors.append('gray')
    else:
        edge_styles.append('solid')
        edge_colors.append('black')

# Generate layout without fixed nodes
pos = nx.spring_layout(G, k=2.0, scale=10, seed=42)

# Dynamically offset each label above the node
import math
node_size_map = {node: size for node, size in zip(G.nodes(), node_sizes)}
offset_scale = 0.02  # Tune as needed for your plot's visual scale
label_pos = {
    node: (x, y + math.sqrt(node_size_map[node]) * offset_scale)
    for node, (x, y) in pos.items()
}

# Plotting
plt.figure(figsize=(14, 14))
# Draw edges individually by style for dashed/solid with correct width and color
for (u, v), style, width, color in zip(G.edges, edge_styles, edge_widths, edge_colors):
    nx.draw_networkx_edges(
        G, pos,
        edgelist=[(u, v)],
        style=style,
        width=width,
        edge_color=color,
        alpha=0.5,  # Try 0.05 to 0.25 for dense graphs
    )
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='white', edgecolors='black', linewidths=2)
nx.draw_networkx_labels(G, label_pos)

# Choose prevalence values to illustrate (e.g., 50%, 25%, 10%)
legend_prevalence = [0.5, 0.25, 0.10]
legend_sizes = [n * 2000 for n in legend_prevalence]

# Coordinates where to plot the legend (tune for your plot's scale)
base_x = 0.02  # fraction of the figure
base_y = 0.02
dy = 0.07

for i, (p, s) in enumerate(zip(legend_prevalence, legend_sizes)):
    plt.scatter([], [], s=s, c='white', edgecolors='black', linewidths=2, label=f"{int(p*100)}%")
# Place the legend at lower left
plt.legend(
    scatterpoints=1,
    frameon=False,
    labelspacing=1.5,
    loc='lower left',     # ensures legend is at the bottom left of the graph
    title="Prevalence",
    fontsize=12,
    title_fontsize=13
)
plt.axis('off')
plt.show()

In [ ]:
from itertools import combinations
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import fisher_exact
import math

def plot_one_lca_subgroup(
    df, class_id, node_color, comorbidity_cols,
    node_size_scale=2000, edge_weight_scale=60, offset_scale=0.055,
    legend_prevalence=(0.5, 0.25, 0.1)
):
    # Subset for LCA class
    lca_sub = df[df['latent_class'] == class_id]
    N = len(lca_sub)
    prevalence = {d: lca_sub[d].sum() / N for d in comorbidity_cols}

    # Build network
    G = nx.Graph()
    for d in comorbidity_cols:
        G.add_node(d, prevalence=prevalence[d])
    for a, b in combinations(comorbidity_cols, 2):
        col_a, col_b = lca_sub[a], lca_sub[b]
        n11 = ((col_a == 1) & (col_b == 1)).sum()
        n10 = ((col_a == 1) & (col_b == 0)).sum()
        n01 = ((col_a == 0) & (col_b == 1)).sum()
        n00 = ((col_a == 0) & (col_b == 0)).sum()
        table = [[n11, n10], [n01, n00]]
        _, pval = fisher_exact(table, alternative='greater')
        if n11 > 0 and pval < 0.05:
            G.add_edge(a, b, weight=n11/N)

    # --- Node and edge visual parameters ---
    node_sizes = [prevalence[n] * node_size_scale for n in G.nodes]
    edge_weights = [G[u][v]['weight'] * edge_weight_scale for u, v in G.edges]
    plt.figure(figsize=(14, 14))
    pos = nx.spring_layout(G, k=1, scale=10, iterations=250, seed=42)

    # --- Edges: thicker and more visible ---
    nx.draw_networkx_edges(G, pos, width=edge_weights, edge_color='gray', alpha=0.6)

    # --- Nodes ---
    nx.draw_networkx_nodes(
        G, pos, node_size=node_sizes, node_color=node_color, edgecolors='black', linewidths=2, alpha=1
    )

    # --- Labels with larger offset ---
    # Radially offset labels:
    center_x, center_y = np.mean([x for x, y in pos.values()]), np.mean([y for x, y in pos.values()])
    node_size_map = {node: size for node, size in zip(G.nodes(), node_sizes)}
    offset = 0.048
    label_pos = {}
    for node, (x, y) in pos.items():
        dx, dy = x - center_x, y - center_y
        r = math.sqrt(node_size_map[node])
        dist = np.linalg.norm([dx, dy]) or 1
        label_pos[node] = (x + dx/dist*r*offset, y + dy/dist*r*offset)
    nx.draw_networkx_labels(G, label_pos, font_size=18)

    # --- Node size legend ---
    legend_sizes = [n * node_size_scale for n in legend_prevalence]
    for p, s in zip(legend_prevalence, legend_sizes):
        plt.scatter([], [], s=s, c=node_color, edgecolors='black', linewidths=2, label=f"{int(p*100)}%")
    plt.legend(
        scatterpoints=1,
        frameon=False,
        labelspacing=1.4,
        loc='lower left',
        title="Prevalence",
        fontsize=14,
        title_fontsize=15
    )
    plt.margins(0.18)
    plt.axis('off')
    plt.title(f"Subgroup {class_id}", fontsize=28)
    plt.tight_layout()
    plt.show()

# Example use for your subgroups (using colors like the paper’s Figure 5)
subgroup_colors = {
    1: "white",
    3: "limegreen",
    4: "blue",
    6: "magenta"
}
for class_id, node_color in subgroup_colors.items():
    plot_one_lca_subgroup(lca_cohort, class_id, node_color, comorbidity_cols)
